<h2 align=center>Facial Expressions Convolutional Neural Network Training</h2>

### 1- Import Libraries

In [22]:
import math
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
import os
from PIL import Image
import glob
from utils import *
import pathlib

### 2- Load and anlyze dataset

In [16]:
class_names = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
for folder in os.listdir("dataset/train/"):
    print(str(len(os.listdir("dataset/train/" + folder))) + " " + folder + " images")

3995 angry images
436 disgust images
4097 fear images
7215 happy images
4965 neutral images
4830 sad images
3171 surprise images


In [17]:
dataset_dir = pathlib.Path("dataset/train/")
total_image_count = len(list(dataset_dir.glob('*/*.jpg')))
print("Total dataset size is: " + str(total_image_count))

Total dataset size is: 28709


In [ ]:
X_train, Y_train = load_training_dataset()
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))

plt.figure(figsize=(10, 10))
idx = np.random.choice(np.arange(len(X_train)), 9, replace=False)
for i, arr_index in enumerate(idx):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(X_train[arr_index], cmap='gray')
    plt.title(class_names[np.squeeze(Y_train[arr_index])])

X_train shape: (28709, 48, 48)
Y_train shape: (28709, 1)


In [21]:
Y_train = convert_to_one_hot(Y_train, 7).T
print("First Y_train after one hot: ", Y_train[0])
print("Y _train shape after one hot: ", Y_train.shape)

First Y_train after one hot:  [1. 0. 0. 0. 0. 0. 0.]
Y _train shape after one hot:  (28709, 7)
